## 1.2 Price-Volume Data Availability Analysis

As worked earlier (see [`index_componentes_webscraper`](./index_components_webscraper.ipynb) notebook), Blackrock publishes the _IVV_-ETF holdings on a monthly basis.

### Importing And Authentication 

In order to handle price and volume data of stocks composing S&P500 we work out first the free dataset `WIKI` which is available on the freemium Nasdaq exchange service [Nasdaq Data Link](https://data.nasdaq.com).

After signing up to get a free account, we can set the necessary steps for downloading the aforementioned dataset:

In [ ]:
#!pip install nasdaq-data-link
# add to environment.yml
# template https://github.com/binder-project/example-conda-environment/blob/master/environment.yml

In [1]:
## 1.2 Price-Volume Data
# Importing And Authentication 
import nasdaqdatalink
import pandas as pd
import datetime as dt
nasdaqdatalink.read_key(filename="data/.my_nq_apikey")

1.  First, let we recover initial and final sample date by loading `pickle` file which contains index components:

In [2]:
IVV = pd.read_pickle("data/pkl/IVV_historical_components_20220727.pkl")
first_components_date = IVV.index[0]
# we substract one-year to start construction of RPMs from this point (we assume static composition from then)
first_date = first_components_date-dt.timedelta(days=365) 
first_date = first_date.strftime("%Y-%m-%d")

"""
https://data.nasdaq.com/databases/WIKIP/documentation?anchor=overview
Note: As of April 11, 2018 this data feed is no longer actively supported by the Quandl 
community. We will continue to host this data feed on Quandl, 
but we do not recommend using it for investment or analysis.
"""
wikip_last_quarter_supported = "2018-03-31" 

nasdaqdatalink.export_table('WIKI/PRICES', 
    qopts = {'columns': ['ticker', 'date', 'adj_close', 'adj_volume']}, 
    date = {'gte': first_date, 'lte': wikip_last_quarter_supported},
    filename = "data/wikip_price_volume_data.zip"
)

In [3]:
wikip_data = pd.read_csv("data/wikip_price_volume_data.zip", parse_dates=True)
wikip_data.head()

,ticker,date,adj_close,adj_volume
0,A,2005-09-29,22.288803,6137100.0
1,A,2005-09-30,22.343382,4864900.0
2,A,2005-10-03,22.507120,4766700.0
3,A,2005-10-04,22.145533,3431700.0
4,A,2005-10-05,21.326844,3914400.0


In [4]:
wikip_data.tail()

,ticker,date,adj_close,adj_volume
8461321,ZUMZ,2018-03-21,23.95,354092.0
8461322,ZUMZ,2018-03-22,23.35,269607.0
8461323,ZUMZ,2018-03-23,23.55,301584.0
8461324,ZUMZ,2018-03-26,24.65,375320.0
8461325,ZUMZ,2018-03-27,23.60,403884.0


2. Let us separate the `wikip_data` by ticker symbol

In [15]:
# Preparing for individual CSV files 
wikip_dict = dict(tuple(wikip_data.groupby("ticker")))
#wikip_dict["AAPL"]
for ticker in wikip_dict.keys():
    wikip_dict[ticker].set_index("date", inplace=True)


In [16]:
wikip_dict["AAPL"]

,ticker,close,volume
date,,,
2005-09-29,AAPL,52.3400,22744500.0
2005-09-30,AAPL,53.6100,18986900.0
2005-10-03,AAPL,54.4400,18126900.0
2005-10-04,AAPL,53.7500,19266400.0
2005-10-05,AAPL,52.7801,21813200.0
...,...,...,...
2018-03-21,AAPL,171.2700,35247358.0
2018-03-22,AAPL,168.8450,41051076.0
2018-03-23,AAPL,164.9400,40248954.0


3. Next, let us find which symbols of the time series index composition is not found on `WIKIP` database

In [30]:
print(f"# Tickers supported by WIKIP: {len(wikip_dict.keys())}")

missing_tickers = []
for row in range(0,len(IVV.index)-1):
    df_tmp = pd.DataFrame(IVV.iloc[row, 0],
        columns=["ticker","company"])
    df_tmp.index = df_tmp["ticker"]
    df_tmp.drop(columns=["ticker"], inplace=True)
    
    stock_components = df_tmp.index.unique()
    for stock in stock_components:
        if stock not in wikip_dict:
            missing_tickers.append((stock, df_tmp.loc[stock,"company"]))

print(f"# Tickers not found in WIKIP: {len(set(missing_tickers))}")

# Tickers supported by WIKIP: 3199
# Tickers not found in WIKIP: 296


Exporting to `csv` file for reference or further use:

In [22]:
wikip_missing_tickers = pd.DataFrame(missing_tickers, columns=["ticker","company"])
wikip_missing_tickers.drop_duplicates(inplace=True)
wikip_missing_tickers.to_csv("data/wikip_missing_tickers.csv", index=False)
wikip_missing_tickers.head(25)

,ticker,company
0,ANDW,ANDREW CORP.
1,PGL,PEOPLES ENERGY CORPORATION
2,ADCT,ADC TELECOMMUNICATIONS
3,HPC,HERCULES INC.
4,GAS.2,NICOR INC
5,PMTC,PARAMETRIC TECHNOLOGY CORP.
6,NOVL,NOVELL
7,DJ,DOW JONES & COMPANY INC.
8,TEK,TEKTRONIX INC.
9,RSHCQ,RS LEGACY CORP


4. Let us find out which of the `WIKIP` unsupported tickers, are supported by _Y! Finance_ API instead.

In [ ]:
#!pip install yfinance
# add environment.yml
import yfinance as yf

In [25]:
# Listed Stock behaviour
yf.Ticker("AAPL").info['regularMarketPrice'] #Apple Stock

161.6295

In [27]:
# Delisted(Unavailable) Stock behaviour
yf.Ticker("LEH").info['regularMarketPrice']==None # The infamous Lehman Brothers

True

In [29]:
len(wikip_missing_tickers["ticker"].unique().tolist())

287

In [32]:
from tqdm.auto import tqdm

yf_missing_tickers = []
for ticker in tqdm(wikip_missing_tickers["ticker"].unique()):
    flag = (yf.Ticker(ticker=ticker).info["regularMarketPrice"]==None)
    if flag:
        print(f"Ticker {ticker} not Found: Appending...")
        yf_missing_tickers.append(ticker)
len(set(yf_missing_tickers))

  0%|          | 0/287 [00:00<?, ?it/s]

Ticker ANDW not Found: Appending...
Ticker PGL not Found: Appending...
Ticker HPC not Found: Appending...
Ticker GAS.2 not Found: Appending...
Ticker PMTC not Found: Appending...
Ticker NOVL not Found: Appending...
Ticker DJ not Found: Appending...
Ticker TEK not Found: Appending...
Ticker RSHCQ not Found: Appending...
Ticker WPI not Found: Appending...
Ticker BOL not Found: Appending...
Ticker AW not Found: Appending...
Ticker SLR not Found: Appending...
Ticker OMX not Found: Appending...
Ticker QLG not Found: Appending...
Ticker TSG not Found: Appending...
Ticker MIL. not Found: Appending...
Ticker NVLS not Found: Appending...
Ticker CCTYQ not Found: Appending...
Ticker UNS1 not Found: Appending...
Ticker APCC. not Found: Appending...
Ticker SBL not Found: Appending...
Ticker HCR not Found: Appending...
Ticker AHEB GR not Found: Appending...
Ticker PTV not Found: Appending...
Ticker FNP not Found: Appending...
Ticker KG not Found: Appending...
Ticker DG.1 not Found: Appending...
Tick

214

In [37]:
yf_missing_tickers = pd.DataFrame(yf_missing_tickers, columns=["ticker"]).drop_duplicates()
yf_missing_tickers.nunique()

ticker    214
dtype: int64

In [38]:
# 214 Tickers Still Missing
yf_missing_tickers.to_csv("data/yf_missing_tickers.csv")

In [53]:
"""
Based upon `yf_missing_tickers.csv` and some handmade web intelligence we
elaborate the final missing data csv file
"""
missing_data = pd.read_csv("data/abnormal_ticker_to_analyze.csv",
index_col="ticker")
missing_data[missing_data["new_ticker"].notna()]

,yfinance_condition,new_ticker
ticker,,
PMTC,ticker_change,PTC
WPI,delisted,ABBV
AW,delisted,RSG
SLR,delisted,FLEX
TSG,ticker_change,SABR
UNS1,ticker_change,VIAV
BFB,ticker_change,BF_B
UUM,ticker_change,UNM
USX1,ticker_change,X


In [54]:
""" 
35 stocks are potentially available from Y! Finance:
 - The most of the companies have changed ticker and long tseries are available.
 - 3 stocks that were delisted can be assimilated to their buyers (IPOs)
"""
ticker_transition_data = missing_data[missing_data["new_ticker"].notna()]
ticker_transition_data["new_ticker"].nunique() 

35

In [57]:
"""
The fund managers of IVV ETF have the mandate to invest more than 90% in S&P500 stock components.
Futures and some cash had also been ocasionally managed
"""
nonstock_tickers = missing_data.loc[missing_data["yfinance_condition"].str.contains("non_stock"),:]
nonstock_tickers

,yfinance_condition,new_ticker
ticker,,
ESZ4,non_stock,NaN
ESH5,non_stock,NaN
ESM5,non_stock,NaN
ESU5,non_stock,NaN
ESZ5,non_stock,NaN
UBFUT,non_stock,NaN
ESH6,non_stock,NaN
ESM6,non_stock,NaN
ESU6,non_stock,NaN


In [59]:
len(nonstock_tickers.index)

33

Therefore:

In [60]:
# Tickers that are not under the WIKIP/YFinance scope
print(yf_missing_tickers.nunique()-(ticker_transition_data["new_ticker"].nunique()+len(nonstock_tickers)) )

ticker    146
dtype: int64


In [76]:
ticker_special_char = yf_missing_tickers[yf_missing_tickers["ticker"].str.contains("\*")]
ticker_special_char["ticker"]

70      FCX*
109     ADM*
117     PRU*
123    AAPL*
129     WFC*
130     AIG*
146     TWX*
153     MOS*
164      FB*
Name: ticker, dtype: object

In [78]:
lst_ticker_special_char = ticker_special_char["ticker"].str.strip("*").tolist()
lst_ticker_special_char

['FCX', 'ADM', 'PRU', 'AAPL', 'WFC', 'AIG', 'TWX', 'MOS', 'FB']

In [80]:
# Some residual missing tickers can also be fixed by dropping special characters
# e.g. FB (also downloadable via META)
for symbol in lst_ticker_special_char:
    yf.download(symbol, start="2022-7-28", end="2022-7-29")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TWX: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [106]:
historical_sp500_tickers_set = set()

for row in tqdm(range(0,len(IVV.index)-1)):
    df_tmp = pd.DataFrame(IVV.iloc[row, 0],
        columns=["ticker","company"])
    lst_running_set = df_tmp["ticker"].tolist()
    historical_sp500_tickers_set.update(lst_running_set)


  0%|          | 0/185 [00:00<?, ?it/s]

In [110]:
print(historical_sp500_tickers_set)

{'CBH', 'ROL', 'FHN', 'FRX', 'ESM0', 'BCR', 'ETFC', 'HFC', 'FCX*', 'NWS', 'RX', 'GGP', 'CF', 'CCE', 'L', 'TDY', 'WDC', 'CTSH', 'JWN', 'DRI', 'BMET', 'CHD', 'KDP', 'APOL', 'SNI', 'RMD', 'PBI', 'XRAY', 'ANSS', 'XOM', 'NXPI', 'KMI.1', 'GLW', 'JNJ', 'NTH', 'SYK', 'VZ', 'ESH0', 'MWV', 'MSFT', 'VMC', 'SPG', 'J', 'K', 'LRCX', 'STJ', 'AAPL', 'DISCK', 'SBAC', 'MHFI', 'ESM8', 'UVN', 'NTAP', 'ESM1', 'TSG', 'CHTR', 'PTV', 'JNPR', 'ESRX', 'DO', 'NOW', 'LUV', 'CPPRQ', 'MRNA', 'NOVL', 'CME', 'EMR', 'DTV', 'SUN', 'ISRG', 'TMO', 'SEDG', 'CPWR', 'DEN', 'AIV', 'MAC', 'SLG', 'HOG', 'ABC', 'LNT', 'DOV', 'AAP', 'AV.1', '3EC', 'WTW', 'WB', 'GNW', 'CMCSK', 'VRSN', 'IR', 'TWX*', '6COP', 'AAL', 'RE', 'ON', 'DPZ', 'BNI', 'NVLS', 'CA', 'KIM', 'ESM6', 'MBI', 'JCI', 'CAM', 'BF.B', 'EMC', 'FFIV', 'APD', 'CPRT', 'FOXA', 'FOSL', 'STI', 'ESU6', 'PCS', 'V', 'ACS', 'NE', 'ADP', 'SHLDQ', 'FO', 'BOL', 'ETR', 'CEPH', 'CLF', 'BLD WI', 'AMZN', 'DELL', 'XRX', 'CC WI', 'TXT', 'F', 'SNV', 'THC', 'WYE', 'HON', 'CSC', 'UBSFT', 'CF

In [112]:
print(f"# Historical Tickers composing S&P500 cached from BlackRock public information: {len(historical_sp500_tickers_set)}")

# Historical Tickers composing S&P500 cached from BlackRock public information: 1008


After all we have a reasonable minimum ~90% of tickers at disposal for the entire sample under consideration. However some further analysis

In [3]:
print(f"{100 - 100* 140 / len(historical_sp500_tickers_set)}% tickers all-times supported")